In [1]:
import sys
sys.path.append('../personal_pkgs/')
import pickle
import pandas as pd
import oop_func as func
import personal_pkg as ref
import parsing_func as pars
import os
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Markdown

In [2]:
fm_base_df = pd.read_csv('file/fm_base_df3.csv')
f_base_df = pd.read_csv('file/f_base_df3.csv')
sc_base_df = pd.read_csv('file/sc_base_df3.csv')
dr_base_df = pd.read_csv('file/dr_base_df3.csv')
base_df = pd.read_csv('file/base_df3.csv')

저널은 similarity score로 merging 하지 않는다.(어느정도 많은 frequency 를 확보하고 있기 때문이다.) 다만, journal 데이터 자체가 깔끔하지 않기 때문에, 정규식으로 preprocessing 을 해준다. 또한, 2009년을 기준으로 한국증권학회지가 AJFS 와 분리되었으므로, 이에 대한 처리를 해준다.

In [3]:
def tuning_the_journal(df) :
#     testing_df = df[df['journal'].notnull()]
    journal_ls = [i for i in df['journal'].tolist()]
    journal_ls1 = [i.replace('”','') if type(i) != float else i for i in journal_ls]
    journal_ls2 = [re.sub('제\s*[0-9]{,5}\s*권|제\s*[0-9]{,5}\s*호|제\s*[0-9]{,5}\s*집','',i) \
                   if type(i) == str else i for i in journal_ls1]
    journal_ls3 = [','.join(re.findall('[a-zA-Z가-힣\s]',i)).replace(",",'').strip().lower() \
                   if type(i) == str else i for i in journal_ls2]
    journal_ls4 = [re.sub('vol$|no$|vol no','',i).strip() \
                   if type(i) == str else i for i in journal_ls3]
    journal_ls5 = [i[:-2].strip() if type(i) != float and re.search('\s[a-z]{1}',i[-2:]) else i for i in journal_ls4]
    df['journal'] = journal_ls5
    df['year'] = df['year'].apply(lambda x : str(x).split('.')[0])
    return df

In [4]:
fm_base_df = tuning_the_journal(fm_base_df)
f_base_df = tuning_the_journal(f_base_df)
sc_base_df = tuning_the_journal(sc_base_df)
dr_base_df = tuning_the_journal(dr_base_df)
base_df = tuning_the_journal(base_df)

In [5]:
def adjust_the_journal_under_2009(base_df) : 
    base_df.reset_index(drop=True,inplace=True)
    testing_df = base_df.copy()

    for idx in range(len(testing_df)) :
        if type(testing_df['journal'][idx]) != float :  
            if ('asiapacific journal of financial studies' in testing_df['journal'][idx]) or \
            ('한국증권학회지' in testing_df['journal'][idx]) or ('증권학회지' in testing_df['journal'][idx]) : 
                try : 
                    if int(testing_df['year'][idx]) < 2009 : 
                        testing_df['journal'][idx] = '한국증권학회지'
                except : testing_df['journal'][idx] = '한국증권학회지'


    for idx in range(len(testing_df)) :
        if type(testing_df['journal'][idx]) != float : 
            if ('한국증권학회지' in testing_df['journal'][idx]) or ('증권학회지' in testing_df['journal'][idx]) : 
                try : 
                    if int(testing_df['year'][idx]) > 2009 : 
                        testing_df['journal'][idx] = '한국증권학회지'
                except : pass # year 가 nan 인 데이터는 검색결과 2009년 이전의 논문이였다.
    return testing_df

In [6]:
base_df = adjust_the_journal_under_2009(base_df)
fm_base_df = adjust_the_journal_under_2009(fm_base_df)
f_base_df = adjust_the_journal_under_2009(f_base_df)
sc_base_df = adjust_the_journal_under_2009(sc_base_df)
dr_base_df = adjust_the_journal_under_2009(dr_base_df)

In [7]:
def fine_revising_the_journal(df):
    a_idx_ls = \
    [idx for idx,i in enumerate(df['journal']) if type(i) != float and ' a' in i[-3:]]
    b_idx_ls = \
    [idx for idx,i in enumerate(df['journal']) if type(i) != float and ' b' in i[-3:]]
    a_or_b_idx_ls = a_idx_ls + b_idx_ls
    
    df.loc[a_or_b_idx_ls,'journal'] = df.loc[a_or_b_idx_ls,'journal'].apply(lambda x : x[:-1].strip())

    slicing_vol_idx_ls = [idx for idx,i in enumerate(df['journal']) if type(i) != float and ' vol' in i]
    df.loc[slicing_vol_idx_ls,'journal'] = df.loc[slicing_vol_idx_ls,'journal'].apply(lambda x : x[:x.find('vol')].strip())
    
    return df

In [8]:
base_df = fine_revising_the_journal(base_df)
fm_base_df = fine_revising_the_journal(fm_base_df)
f_base_df = fine_revising_the_journal(f_base_df)
sc_base_df = fine_revising_the_journal(sc_base_df)
dr_base_df = fine_revising_the_journal(dr_base_df)

한국 학회인데, 영어로 기제된 경우, 이를 한글로 모두 통일시켜 준다.

In [9]:
revise_dict = {'Korean Journal of Financial Stuides' : '한국증권학회지',
'Korean Journal of Financial Engineering':'금융공학연구',
'Korean Journal of Futures and Options':'선물연구',
'Asian Review of Financial Research':'재무연구',
'KOREAN JOURNAL OF FINANCIAL MANAGEMENT':'재무관리연구',
'Korean Management Review':'경영학연구',
'Journal of Money & Finance':'금융연구',
'Korean Journal of Money & Finance':'금융학회지',
'Korean Accounting Review':'회계학연구',
'Korean Journal of Business Administration':'대한경영학회지',
'Journal of Industrial Economics and Business':'산업경제연구',
'International Business Journal':'국제경영연구',
'ACCOUNTING INFORMATION REVIEW':'회계정보연구',
'Korean Accounting Journal':'회계저널',
}

In [10]:
def convert_eng_journal_name_to_kor_journal_name(df,revise_dict):
    for index in range(len(revise_dict)) : 
        revise_ls = \
        [idx for idx,i in enumerate(df['journal']) \
         if type(i) != float and list(revise_dict.keys())[index].lower() in i]

        df.loc[revise_ls,'journal'] = list(revise_dict.values())[index]
    
    return df

In [11]:
base_df = convert_eng_journal_name_to_kor_journal_name(base_df,revise_dict)
fm_base_df = convert_eng_journal_name_to_kor_journal_name(fm_base_df,revise_dict)
f_base_df = convert_eng_journal_name_to_kor_journal_name(f_base_df,revise_dict)
sc_base_df = convert_eng_journal_name_to_kor_journal_name(sc_base_df,revise_dict)
dr_base_df = convert_eng_journal_name_to_kor_journal_name(dr_base_df,revise_dict)

In [12]:
def make_counter_df_under_journal(base_df) :
    df1 = pd.DataFrame.from_dict(Counter(base_df['journal'].tolist()),orient='index',columns=['count'])
    df2 = df1.sort_values('count',ascending=False)
    return df2

In [13]:
journal_counter_df = make_counter_df_under_journal(base_df)
journal_counter_df.reset_index(inplace=True)
display(journal_counter_df.head(10))
journal_counter_df.to_csv('journal_counter_df.csv',index=False)

,index,count
0,journal of finance,3782
1,journal of financial economics,3130
2,한국증권학회지,1612
3,review of financial studies,1029
4,재무관리연구,951
5,재무연구,931
6,econometrica,555
7,journal of political economy,546
8,american economic review,517
9,선물연구,501


In [14]:
display(Markdown('### 한국재무관리학회'))
display(make_counter_df_under_journal(fm_base_df).head(10))
display(Markdown('### 한국재무학회'))
display(make_counter_df_under_journal(f_base_df).head(10))
display(Markdown('### 한국증권학회지'))
display(make_counter_df_under_journal(sc_base_df).head(10))
display(Markdown('### 한국파생상품학회'))
display(make_counter_df_under_journal(dr_base_df).head(10))

### 한국재무관리학회

,count
journal of finance,1390
journal of financial economics,1084
재무관리연구,429
한국증권학회지,410
review of financial studies,390
재무연구,274
journal of financial and quantitative analysis,230
econometrica,206
american economic review,199
journal of banking and finance,169


### 한국재무학회

,count
journal of finance,895
journal of financial economics,704
한국증권학회지,330
재무연구,278
review of financial studies,238
재무관리연구,210
econometrica,136
american economic review,129
journal of political economy,109
journal of financial and quantitative analysis,108


### 한국증권학회지

,count
journal of finance,1205
journal of financial economics,1147
한국증권학회지,760
재무연구,320
review of financial studies,259
재무관리연구,238
journal of political economy,229
american economic review,154
econometrica,129
회계학연구,91


### 한국파생상품학회

,count
journal of finance,301
선물연구,236
review of financial studies,182
journal of financial economics,175
한국증권학회지,103
econometrica,81
재무관리연구,74
journal of futures markets,65
journal of financial and quantitative analysis,61
재무연구,58


In [15]:
fm_base_df.to_csv('file/fm_base_df3.csv')
f_base_df.to_csv('file/f_base_df3.csv')
sc_base_df.to_csv('file/sc_base_df3.csv')
dr_base_df.to_csv('file/dr_base_df3.csv')
base_df.to_csv('file/base_df3.csv')